In [153]:
from langchain_openai.llms import OpenAI

llm = OpenAI()


In [154]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Responda a seguinte pergunta do usuário
{pergunta}
""")

In [155]:
print(prompt_template.format(pergunta="O que é um SaaS?"))


Responda a seguinte pergunta do usuário
O que é um SaaS?



In [156]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
{pergunta}
""")

In [157]:
print(prompt_template.format(pergunta="O que é um SaaS?", n_palavras=15))


Responda a seguinte pergunta do usuário em até 15 palavras:
O que é um SaaS?



In [158]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
{pergunta}
""", partial_variables={"n_palavras": 10}
)

In [159]:
print(prompt_template.format(pergunta="O que é LangChain?"))


Responda a seguinte pergunta do usuário em até 10 palavras:
O que é LangChain?



### Utilizando Múltiplos Prompts

In [160]:
from langchain.prompts import PromptTemplate

template_word_count = PromptTemplate.from_template("""
Responda a pergunta em até {n_palavras} palavras:
""")

template_line_count = PromptTemplate.from_template("""
Responda a pergunta em até {n_linhas} linhas:

""")

template_idiom = PromptTemplate.from_template("""
Retorne a resposta em {idiom}.

""")

template_final = (template_word_count + template_line_count + template_idiom +
                  "Responda a pergunta seguindo as instruções {pergunta}")

In [161]:
print(template_final)

input_variables=['idiom', 'n_linhas', 'n_palavras', 'pergunta'] template='\nResponda a pergunta em até {n_palavras} palavras:\n\nResponda a pergunta em até {n_linhas} linhas:\n\n\nRetorne a resposta em {idiom}.\n\nResponda a pergunta seguindo as instruções {pergunta}'


In [162]:
prompt_final = template_final.format(n_palavras=15, n_linhas = 1, idiom="inglês", pergunta="O que é o Sol?")
llm.invoke(prompt_final)

'\n\nO Sol é a estrela central do sistema solar que fornece luz e calor para a Terra. (em inglês: The Sun is the central star of the solar system that provides light and heat to Earth.)'

In [163]:
print(prompt_final)


Responda a pergunta em até 15 palavras:

Responda a pergunta em até 1 linhas:


Retorne a resposta em inglês.

Responda a pergunta seguindo as instruções O que é o Sol?


### Templates para Chat

In [164]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template("Essa é a minha dúvida: {duvida}")
chat_template.format_messages(duvida="Quem é você?")

[HumanMessage(content='Essa é a minha dúvida: Quem é você?')]

In [165]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente irônico e se chama {nome_assistente}"),
        ("human", "Olá como vai?"),
        ("ai", "Estou bem, como posso lhe ajudar?"),
        ("human", "{pergunta}")
    ]
)

In [166]:
chat_template

ChatPromptTemplate(input_variables=['nome_assistente', 'pergunta'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['nome_assistente'], template='Você é um assistente irônico e se chama {nome_assistente}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Olá como vai?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Estou bem, como posso lhe ajudar?')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pergunta'], template='{pergunta}'))])

In [167]:
chat_template.format_messages(nome_assistente="BotX", pergunta="Qual seu nome?")

[SystemMessage(content='Você é um assistente irônico e se chama BotX'),
 HumanMessage(content='Olá como vai?'),
 AIMessage(content='Estou bem, como posso lhe ajudar?'),
 HumanMessage(content='Qual seu nome?')]

In [168]:
from langchain_openai.chat_models import ChatOpenAI
chat=ChatOpenAI()

chat.invoke(chat_template.format_messages(nome_assistente="BotX", pergunta="Qual seu nome?"))

AIMessage(content='Meu nome é BotX, estou aqui para ajudar com o que você precisar.', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 54, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5d67bd66-dfda-4d67-ae80-29bf87972b22-0')

### Few Shot Prompting

In [169]:
exemplos = [
    {"pergunta": "Qual é a maior montanha do mundo, o Monte Everest ou o K2?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Qual é a altura do Monte Everest? 
Resposta intermediária: O Monte Everest tem 8.848 metros de altura. 
Pergunta de acompanhamento: Qual é a altura do K2? 
Resposta intermediária: O K2 tem 8.611 metros de altura. 
Então a resposta final é: Monte Everest 
""", 
    }, 
    {"pergunta": "Quem nasceu primeiro, Charles Darwin ou Albert Einstein?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quando nasceu Charles Darwin? 
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809. 
Pergunta de acompanhamento: Quando nasceu Albert Einstein? 
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879. 
Então a resposta final é: Charles Darwin 
""", 
    }, 
    {"pergunta": "Quem foi o pai de Napoleão Bonaparte?",
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi Napoleão Bonaparte? 
Resposta intermediária: Napoleão Bonaparte foi um líder militar e imperador francês. 
Pergunta de acompanhamento: Quem foi o pai de Napoleão Bonaparte? 
Resposta intermediária: O pai de Napoleão Bonaparte foi Carlo Buonaparte. 
Então a resposta final é: Carlo Buonaparte 
""", 
    },
    {"pergunta": "Os filmes 'O Senhor dos Anéis' e 'O Hobbit' foram dirigidos pelo mesmo diretor?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem dirigiu 'O Senhor dos Anéis'? 
Resposta intermediária: 'O Senhor dos Anéis' foi dirigido por Peter Jackson. 
Pergunta de acompanhamento: Quem dirigiu 'O Hobbit'? 
Resposta intermediária: 'O Hobbit' também foi dirigido por Peter Jackson. 
Então a resposta final é: Sim 
""",
    },
]

In [170]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate


example_prompt = PromptTemplate(
    input_variables=["pergunta", "resposta"],
    template="Pergunta {pergunta}\n{resposta}"
)

example_prompt.format(**exemplos[0])

'Pergunta Qual é a maior montanha do mundo, o Monte Everest ou o K2?\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Qual é a altura do Monte Everest? \nResposta intermediária: O Monte Everest tem 8.848 metros de altura. \nPergunta de acompanhamento: Qual é a altura do K2? \nResposta intermediária: O K2 tem 8.611 metros de altura. \nEntão a resposta final é: Monte Everest \n'

In [171]:
prompt = FewShotPromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt,
    suffix="Pergunta: {input}",
    input_variables=["input"]
)

In [177]:
llm.invoke(prompt.format(input="Quais são os métodos que uma classe recebe?"))

'\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: O que é uma classe? \nResposta intermediária: Uma classe é um conjunto de atributos e métodos que definem um objeto em programação orientada a objetos. \nPergunta de acompanhamento: Como uma classe recebe métodos? \nResposta intermediária: Uma classe recebe métodos através da herança de outras classes ou da implementação de interfaces. \nEntão a resposta final é: Uma classe pode receber métodos através da herança ou da implementação de interfaces.'

In [ ]:
llm.invoke(prompt.format(input="como Uma classe recebe métodos através da herança de outras classes ou da implementação de interfaces, EXPLIQUE"))mn 


'?\nSão necessárias perguntas de acompanhamento aqui: Não. \nResposta: Quando uma classe herda de outra classe, ela recebe todos os métodos públicos e protegidos da classe pai. Além disso, uma classe pode implementar uma ou mais interfaces, o que significa que ela deve fornecer a implementação para todos os métodos definidos nessas interfaces. Isso permite que uma classe tenha comportamentos e funcionalidades adicionais sem precisar reescrever o código para essas funcionalidades. '